<a href="https://colab.research.google.com/github/Ralfs-GitHub/100-days-of-machine-learning/blob/main/Rakuten_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Modules

- TorchMetrics is a collection of 100+ PyTorch metrics implementations and an easy-to-use API to create custom metrics. It offers: A standardized interface to increase reproducibility.

-The Python "re" module provides regular expression support. In Python a regular expression search is typically written as: match = re. search(pat, str) The re.search() method takes a regular expression pattern and a string and searches for that pattern within the string.

In [70]:
pip install torchmetrics -q

In [71]:
import pandas as pd
import numpy as np
import os
import re
import copy
import time
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from wordcloud import WordCloud

import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import torch
import torch.nn as nn
from torchvision import models
from transformers import BertModel, BertTokenizer
from torchmetrics.classification import MulticlassF1Score
import torch.nn.functional as F

from tensorflow.keras.preprocessing.image import load_img, array_to_img
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [72]:
torch.cuda.set_device(torch.cuda.current_device()) if torch.cuda.is_available() else print("CUDA is not available. Using CPU.")
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

## Load Dataset

In [73]:
BASE_PATH = "/content/drive/MyDrive/rakuten/"

In [74]:
!unzip -q "/content/drive/MyDrive/rakuten/images.zip"

replace images/image_test/image_1276746131_product_4015997768.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [77]:
train = pd.read_csv(BASE_PATH+'X_train_update.csv')
temp = pd.read_csv(BASE_PATH+"Y_train_CVw08PX.csv")
train['prdtypecode'] = temp['prdtypecode']
train = train.drop(columns=['Unnamed: 0'], axis=1)
# train = train[:5000]
train.head()

ParserError: ignored

In [78]:
# create image path to access images
train['image_path'] = train.apply(lambda x: f"images/image_train/image_{x['imageid']}_product_{x['productid']}.jpg", axis=1)
train.head()

NameError: ignored

In [79]:
train.describe()

NameError: ignored

In [80]:
train.isnull().sum()

NameError: ignored

In [81]:
train.info()

NameError: ignored

In [82]:
train['prdtypecode'].value_counts()

NameError: ignored

In [83]:
# total output classes in the dataset
train['prdtypecode'].nunique()

NameError: ignored

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Exploratory Data Analysis

In [85]:
image = Image.open(train['image_path'][0])
print(train['designation'][0])
image

NameError: ignored

In [86]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [87]:
image.size

NameError: ignored

In [34]:
sns.countplot(y='prdtypecode', data=train)

NameError: ignored

In [88]:
# to display grid of images (7x7)
plt.figure(figsize=(20, 20))
temp_df = train.sample(25).reset_index(drop=True)
index = 1

for i in range(25):
    plt.subplot(5, 5, index)
    # load the image
    img = load_img(temp_df['image_path'][i])
    # convert to numpy array
    img = np.array(img)
    # show the image
    plt.title("Product Code: "+str(temp_df['prdtypecode'][i]))
    plt.imshow(img)
    plt.axis('off')
    # increment the index for next image
    index += 1

NameError: ignored

<Figure size 2000x2000 with 0 Axes>

## Data Preprocessing

In [89]:
nltk.download('punkt')
nltk.download('stopwords')

stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))

def clean_text(text, for_embedding=False):
    """
        - remove any html tags (< /br> often found)
        - Keep only ASCII + European Chars and whitespace, no digits
        - remove single letter chars
        - convert all whitespaces (tabs etc.) to single wspace
        if not for embedding (but e.g. tdf-idf):
        - all lowercase
        - remove stopwords, punctuation and stemm
    """
    RE_WSPACE = re.compile(r"\s+", re.IGNORECASE)
    RE_TAGS = re.compile(r"<[^>]+>")
    RE_ASCII = re.compile(r"[^A-Za-zÀ-ž ]", re.IGNORECASE)
    RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž]\b", re.IGNORECASE)
    if for_embedding:
        # Keep punctuation
        RE_ASCII = re.compile(r"[^A-Za-zÀ-ž,.!? ]", re.IGNORECASE)
        RE_SINGLECHAR = re.compile(r"\b[A-Za-zÀ-ž,.!?]\b", re.IGNORECASE)

    text = re.sub(RE_TAGS, " ", text)
    text = re.sub(RE_ASCII, " ", text)
    text = re.sub(RE_SINGLECHAR, " ", text)
    text = re.sub(RE_WSPACE, " ", text)

    word_tokens = word_tokenize(text)
    words_tokens_lower = [word.lower() for word in word_tokens]

    if for_embedding:
        # no stemming and punctuation / stop words removal
        words_filtered = words_tokens_lower
    else:
        words_filtered = [
            stemmer.stem(word) for word in words_tokens_lower if word not in stop_words
        ]

    text_clean = " ".join(words_filtered)
    return text_clean

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [90]:
train["designation_clean"] = train["designation"].map(lambda x: clean_text(x, for_embedding=True) if isinstance(x, str) else x)

NameError: ignored

In [91]:
train.head()

NameError: ignored

In [92]:
# max designation length
max(train['designation_clean'].apply(lambda x: len(x.split())))

NameError: ignored

In [93]:
# collect all tokens
all_tokens = " ".join(train['designation_clean']).split()

# Remove stop words
stemmer = SnowballStemmer("german")
stop_words = set(stopwords.words("german"))
filtered_tokens = [word for word in all_tokens if word.isalnum() and word not in stop_words]

stemmed_tokens = [stemmer.stem(word) for word in filtered_tokens]

# Calculate word frequencies
word_freq = Counter(stemmed_tokens)

NameError: ignored

In [94]:
# Visualize the top N words
top_words = 20
common_words = word_freq.most_common(top_words)
labels, values = zip(*common_words)

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(labels, values, color='skyblue')
plt.title(f"Top {top_words} Words Frequency")
plt.xlabel("Words")
plt.xticks(rotation=45)
plt.ylabel("Frequency")
plt.show()

NameError: ignored

In [95]:
# Generate the word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(" ".join(stemmed_tokens))

# Display the word cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')  # Turn off axis labels
plt.title('Word Cloud for Product Descriptions')
plt.show()

NameError: ignored

## Data Preparation for Model Training

In [96]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

train['prdtypecode'] = le.fit_transform(train['prdtypecode'])

NameError: ignored

In [97]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(train, test_size=0.2, random_state=42, stratify=train['prdtypecode'])

NameError: ignored

In [98]:
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

NameError: ignored

In [99]:
class RakutenDataset(Dataset):
    def __init__(self, df, transform=None, max_text_length=50):
        self.image_paths = df['image_path']
        self.text_data = df['designation_clean']
        self.labels = df['prdtypecode']
        self.transform = transform
        self.max_text_length = max_text_length
        self.tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-german-uncased')
        self.num_classes = 27

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        text = self.text_data[idx]
        label = self.labels[idx]

        # Load and transform the image
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Tokenize the text
        text_tokens = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_text_length,
            return_tensors='pt'
        )

        # One-hot encode the label
        # label_one_hot = F.one_hot(torch.tensor(label), num_classes=self.num_classes)

        return {
            'image': image,
            'text_tokens': text_tokens,
            'label': torch.tensor(label)
        }

In [100]:
# Define transformations for training images
image_transform_train = transforms.Compose([
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0), ratio=(0.8, 1.2)),  # Resize to 224x224
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Adjust color jitter
    transforms.RandomRotation(degrees=15),  # Randomly rotate the image by up to 15 degrees
    transforms.RandomVerticalFlip(),  # Randomly flip the image vertically
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=0.1),  # Random affine transformations
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Define transformations for validation images
image_transform_val = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(), # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

In [101]:
# Create an instance of the dataset
train_dataset = RakutenDataset(train_df, transform=image_transform_train)
val_dataset = RakutenDataset(val_df, transform=image_transform_val)

# Create a DataLoader for the dataset
batch_size = 128
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

data_loaders = {'train': train_loader, 'val': val_loader}

NameError: ignored

In [102]:
train_dataset[0]

NameError: ignored

## Model Creation

In [103]:
class ImageTextClassifier(nn.Module):
    def __init__(self, num_classes, image_model='resnet101', text_model='dbmdz/bert-base-german-uncased'):
        super(ImageTextClassifier, self).__init__()

        # Image processing model (CNN)
        if image_model == 'resnet101':
            self.image_model = models.resnet101(pretrained=True, )
            self.image_model.fc = nn.Identity()  # Remove the final fully connected layer
            image_feature_size = 2048  # Adjust based on the chosen CNN model
        else:
            raise ValueError("Unsupported image model. Choose 'resnet101'.")

        # Text processing model (BERT)
        self.text_model = BertModel.from_pretrained(text_model)
        self.tokenizer = BertTokenizer.from_pretrained(text_model)
        text_feature_size = 768  # BERT output size

        # Freeze the pretrained layers
        for param in self.image_model.parameters():
            param.requires_grad = False
        for param in self.text_model.parameters():
            param.requires_grad = False

        # Fully connected layers for classification
        self.fc1 = nn.Linear(image_feature_size + text_feature_size, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, image, text):
        # Image processing
        image_features = self.image_model(image)

        # Text processing
        input_ids = text['input_ids'].squeeze(1)  # Remove the extra dimension
        attention_mask = text['attention_mask'].squeeze(1)  # Remove the extra dimension

        # Forward pass through BERT
        outputs = self.text_model(input_ids=input_ids, attention_mask=attention_mask)

        # Extract the last hidden state and calculate mean along the sequence length
        last_hidden_state = outputs.last_hidden_state
        text_features = last_hidden_state.mean(dim=1)

        # Concatenate image and text features
        combined_features = torch.cat((image_features, text_features), dim=1)

        # Fully connected layers
        x = torch.relu(self.fc1(combined_features))
        x = self.fc2(x)

        return x

In [104]:
# # Example usage:
# num_classes = 27
# batch_size = 1
# model = ImageTextClassifier(num_classes=num_classes)

# # Create dummy input tensors (adjust size based on your actual input)
# image_input = torch.randn((batch_size, 3, 224, 224))  # RGB image with size 224x224
# text_input = train_dataset['text_tokens']['input_ids'] * batch_size  # List of text inputs

# # Forward pass
# output = model(image_input, text_input)
# print(output.shape)

## Model Training

In [105]:
num_classes = 27
model = ImageTextClassifier(num_classes=num_classes)
model = model.to(DEVICE)

In [56]:
import math
def cyclical_lr(stepsize, min_lr=3e-4, max_lr=3e-3):

    # Scaler: we can adapt this if we do not want the triangular CLR
    scaler = lambda x: 1.

    # Lambda function to calculate the LR
    lr_lambda = lambda it: min_lr + (max_lr - min_lr) * relative(it, stepsize)

    # Additional function to see where on the cycle we are
    def relative(it, stepsize):
        cycle = math.floor(1 + it / (2 * stepsize))
        x = abs(it / stepsize - 2 * cycle + 1)
        return max(0, (1 - x)) * scaler(cycle)

    return lr_lambda

In [106]:
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
optimizer = torch.optim.SGD(model.parameters(), lr=1., momentum=0.9)
step_size = 4*len(train_loader)
clr = cyclical_lr(step_size, min_lr=3e-4, max_lr=3e-3)
scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, [clr])

NameError: ignored

In [107]:
# find total parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters')
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'{trainable_params:,} training parameters')

153,883,995 total parameters
1,456,155 training parameters


In [108]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=5, scheduler=None):
    # set starting time
    start_time = time.time()

    val_f1_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs-1}')
        print('-'*15)

        # each epoch has training and validation phase
        for phase in ['train', 'val']:
            # set mode for model
            if phase == 'train':
                model.train()  # set model to training mode
            else:
                model.eval()  # set model to evaluate mode

            running_loss = 0.0
            fin_out = []
            fin_labels = []

            # iterate over data
            for batch in dataloaders[phase]:
                image_input = batch['image'].to(DEVICE)
                text_input = batch['text_tokens'].to(DEVICE)
                labels = batch['label'].to(DEVICE)

                # reset (or) zero the parameter gradients
                optimizer.zero_grad()

                # training (or) validation process
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(image_input, text_input)
                    # print("Outputs values:", outputs[:5])  # Print the first 5 values
                    # print("Labels values:", labels[:5])   # Print the first 5 values
                    loss = criterion(outputs, labels)

                    # accumulate predictions and labels for F1 score
                    fin_out.append(outputs.cpu().detach())
                    fin_labels.append(labels.cpu().detach())

                    # backpropagation in the network
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        scheduler.step()

                running_loss += loss.item() * image_input.size(0)

            # calculate loss for the epoch
            epoch_loss = running_loss / len(dataloaders[phase].dataset)

            # calculate F1 score for the epoch
            fin_out = torch.cat(fin_out, dim=0)
            fin_labels = torch.cat(fin_labels, dim=0)
            f1_score = MulticlassF1Score(num_classes=fin_out.shape[1], average='weighted')(fin_out, fin_labels)

            # print loss and F1 score for training & validation
            print('{} Loss: {:.4f} F1: {:.4f}'.format(phase, epoch_loss, f1_score))

            # update the best weights
            if phase == 'val' and f1_score > best_f1:
                best_f1 = f1_score
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_f1_history.append(f1_score)

        print()

    end_time = time.time() - start_time

    print('Training completes in {:.0f}m {:.0f}s'.format(end_time // 60, end_time % 60))
    print('Best Val F1 Score: {:.4f}'.format(best_f1))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_f1_history

In [109]:
num_epochs = 10
model, f1_history = train_model(model=model, dataloaders=data_loaders, criterion=criterion, optimizer=optimizer, num_epochs=num_epochs, scheduler=scheduler)

NameError: ignored

In [110]:
# # save the model and model weights
torch.save(model, 'best_model.pt')
torch.save(model.state_dict(), 'best_model_weights')

## Model Testing

In [111]:
tokenizer = BertTokenizer.from_pretrained('dbmdz/bert-base-german-uncased')
def predict_product(df, i):
    # load the image
    img = load_img(df['image_path'][i])
    # convert to numpy array
    img = np.array(img)
    # show the image
    # plt.title("Product Code: "+str(df['prdtypecode'][i]))
    plt.imshow(img)
    plt.axis('off')

    # get features
    # Load and transform the image
    image = Image.open(df['image_path'][i]).convert('RGB')
    image = image_transform_val(image)

    # Tokenize the text
    text_tokens = tokenizer(
        df['designation_clean'][i],
        truncation=True,
        padding='max_length',
        max_length=50,
        return_tensors='pt'
    )

    image = image.unsqueeze(0).to(DEVICE)
    text_tokens = text_tokens.to(DEVICE)

    # model prediction
    model.eval()
    with torch.no_grad():
        outputs = model(image, text_tokens)
    _, preds = torch.max(outputs, 1)

    print("Image Path:", df['image_path'][i])
    print("Product Title:", df['designation'][i])

    print()
    print("Actual Class:",le.inverse_transform([df['prdtypecode'][i]])[0])
    print("Predicted Class:", le.inverse_transform([preds.item()])[0])

In [112]:
# here i represents the index of the dataset
predict_product(val_df, i=5)

NameError: ignored

In [113]:
predict_product(val_df, i=10)

NameError: ignored

In [114]:
model.eval()
from sklearn.metrics import f1_score
def calculate_accuracy(predictions, labels):
    _, predicted = torch.max(predictions, 1)
    correct = (predicted == labels).sum().item()
    total = labels.size(0)
    accuracy = correct / total
    return accuracy

total_accuracy = 0.0
total_samples = 0

all_predictions = []
all_labels = []

# iterate over data
for batch in data_loaders['val']:
    image_input = batch['image'].to(DEVICE)
    text_input = batch['text_tokens'].to(DEVICE)
    labels = batch['label'].to(DEVICE)
    # predict labels
    with torch.no_grad():
        outputs = model(image_input, text_input)

    # calculate accuracy for this batch
    batch_accuracy = calculate_accuracy(outputs, labels)

    # update total accuracy and total samples
    total_accuracy += batch_accuracy * image_input.size(0)
    total_samples += image_input.size(0)

    # convert predictions and labels to numpy arrays
    _, predicted = torch.max(outputs, 1)
    batch_predictions = predicted.cpu().numpy()
    batch_labels = labels.cpu().numpy()

    # append batch predictions and labels to the lists
    all_predictions.extend(batch_predictions)
    all_labels.extend(batch_labels)

# calculate the overall accuracy
overall_accuracy = total_accuracy / total_samples
# calculate f1 score
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f'F1 Score: {f1:.4f}')
print(f'Test Accuracy: {overall_accuracy * 100:.2f}%')

NameError: ignored

In [115]:
from sklearn.metrics import confusion_matrix

# get confusion matrix
conf_matrix = confusion_matrix(all_labels, all_predictions)

# plot the heatmap
plt.figure(figsize=(20, 20))
sns.heatmap(conf_matrix, annot=True, fmt='d')
plt.title(f'Confusion Matrix (F1 Score: {f1:.2f})')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

ValueError: ignored

<Figure size 2000x2000 with 0 Axes>

In [116]:
from sklearn.metrics import classification_report
print(classification_report(all_labels, all_predictions))

ValueError: ignored